# Feature Engineering Extended
## Calculations and recalculations after correcting wind dir differences
<ul><li>Wind direction spreads</li>
<li>Wind direction splits</li></ul>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
# Python libs
import numpy as np
import pandas as pd
from os.path import isfile, join
import os.path
import re
from google.colab import drive
import sys

In [0]:
steps_10_mins = 1
steps_20_mins = 2
steps_30_mins = 3
steps_1_hour = 6
steps_2_hours = 12
steps_12_hours = 12 * steps_1_hour
steps_24_hours = 24 * steps_1_hour

column_year = 'year'
column_month = 'month'
column_hour = 'hour'

MASTER_DATASET = 'model_data_all_features_classified.csv.zip'

In [0]:
pd.options.display.float_format = '{:,.2f}'.format # Limit output to 3 decimal places.
pd.set_option('display.max_columns', None)

In [0]:
data_path = '../gdrive/My Drive/bise_data'

In [6]:
drive.mount('/gdrive')
#drive.mount('/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
file_master = os.path.join(data_path, MASTER_DATASET)
df_master = pd.read_csv(file_master, index_col=0)
df_master.head()

,datetime,original_index,year,month,bise_event,rule3_episode,rule4_episode,rule3_duration,rule4_duration,rule3_label,rule4_label,manual_bise_x,manual_bise_y,manual_duration_class,rule4_episode_1_0,rule4_duration_class,KLO_Air_temp,KLO_Hum,KLO_Hum_delta_3,KLO_Hum_delta_6,KLO_QNH,KLO_Wind_dir,KLO_Wind_dir_delta,KLO_Wind_dir_delta-3,KLO_Wind_dir_delta-6,KLO_Wind_dir_spread30,KLO_Wind_dir_spread60,KLO_Wind_max_EW,KLO_Wind_max_NS,KLO_Wind_max,KLO_Wind_max_delta_3,KLO_Wind_max_delta_6,KLO_Wind_speed_EW,KLO_Wind_speed_NS,KLO_Wind_speed,KLO_Wind_speed_delta_3,KLO_Wind_speed_delta_6,BIZ_Air_temp,BIZ_Hum,BIZ_Hum_delta_3,BIZ_Hum_delta_6,BIZ_QNH,BIZ_Wind_dir,BIZ_Wind_dir_delta,BIZ_Wind_dir_delta-3,BIZ_Wind_dir_delta-6,BIZ_Wind_dir_spread30,BIZ_Wind_dir_spread60,BIZ_Wind_max_EW,BIZ_Wind_max_NS,BIZ_Wind_max,BIZ_Wind_max_delta_3,BIZ_Wind_max_delta_6,BIZ_Wind_speed_EW,BIZ_Wind_speed_NS,BIZ_Wind_speed,BIZ_Wind_speed_delta_3,BIZ_Wind_speed_delta_6,BUS_Air_temp,BUS_Hum,BUS_Hum_delta_3,BUS_Hum_delta_6,BUS_QNH,BUS_Wind_dir,BUS_Wind_dir_delta,BUS_Wind_dir_delta-3,BUS_Wind_dir_delta-6,BUS_Wind_dir_spread30,BUS_Wind_dir_spread60,BUS_Wind_max_EW,BUS_Wind_max_NS,BUS_Wind_max,BUS_Wind_max_delta_3,BUS_Wind_max_delta_6,BUS_Wind_speed_EW,BUS_Wind_speed_NS,BUS_Wind_speed,BUS_Wind_speed_delta_3,BUS_Wind_speed_delta_6,GUT_Air_temp,GUT_Hum,GUT_Hum_delta_3,GUT_Hum_delta_6,GUT_QNH,GUT_Wind_dir,GUT_Wind_dir_delta,GUT_Wind_dir_delta-3,GUT_Wind_dir_delta-6,GUT_Wind_dir_spread30,GUT_Wind_dir_spread60,GUT_Wind_max_EW,GUT_Wind_max_NS,GUT_Wind_max,GUT_Wind_max_delta_3,GUT_Wind_max_delta_6,GUT_Wind_speed_EW,GUT_Wind_speed_NS,GUT_Wind_speed,GUT_Wind_speed_delta_3,GUT_Wind_speed_delta_6,KOP_Air_temp,KOP_Hum,KOP_Hum_delta_3,KOP_Hum_delta_6,KOP_QNH,KOP_Wind_dir,KOP_Wind_dir_delta,KOP_Wind_dir_delta-3,KOP_Wind_dir_delta-6,KOP_Wind_dir_spread30,KOP_Wind_dir_spread60,KOP_Wind_max_EW,KOP_Wind_max_NS,KOP_Wind_max,KOP_Wind_max_delta_3,KOP_Wind_max_delta_6,KOP_Wind_speed_EW,KOP_Wind_speed_NS,KOP_Wind_speed,KOP_Wind_speed_delta_3,KOP_Wind_speed_delta_6,MOA_Air_temp,MOA_Hum,MOA_Hum_delta_3,MOA_Hum_delta_6,MOA_QNH,MOA_Wind_dir,MOA_Wind_dir_delta,MOA_Wind_dir_delta-3,MOA_Wind_dir_delta-6,MOA_Wind_dir_spread30,MOA_Wind_dir_spread60,MOA_Wind_max_EW,MOA_Wind_max_NS,MOA_Wind_max,MOA_Wind_max_delta_3,MOA_Wind_max_delta_6,MOA_Wind_speed_EW,MOA_Wind_speed_NS,MOA_Wind_speed,MOA_Wind_speed_delta_3,MOA_Wind_speed_delta_6,MUB_Air_temp,MUB_Hum,MUB_Hum_delta_3,MUB_Hum_delta_6,MUB_QNH,MUB_Wind_dir,MUB_Wind_dir_delta,MUB_Wind_dir_delta-3,MUB_Wind_dir_delta-6,MUB_Wind_dir_spread30,MUB_Wind_dir_spread60,MUB_Wind_max_EW,MUB_Wind_max_NS,MUB_Wind_max,MUB_Wind_max_delta_3,MUB_Wind_max_delta_6,MUB_Wind_speed_EW,MUB_Wind_speed_NS,MUB_Wind_speed,MUB_Wind_speed_delta_3,MUB_Wind_speed_delta_6,PAY_Air_temp,PAY_Hum,PAY_Hum_delta_3,PAY_Hum_delta_6,PAY_QNH,PAY_Wind_dir,PAY_Wind_dir_delta,PAY_Wind_dir_delta-3,PAY_Wind_dir_delta-6,PAY_Wind_dir_spread30,PAY_Wind_dir_spread60,PAY_Wind_max_EW,PAY_Wind_max_NS,PAY_Wind_max,PAY_Wind_max_delta_3,PAY_Wind_max_delta_6,PAY_Wind_speed_EW,PAY_Wind_speed_NS,PAY_Wind_speed,PAY_Wind_speed_delta_3,PAY_Wind_speed_delta_6,PUY_Air_temp,PUY_Hum,PUY_Hum_delta_3,PUY_Hum_delta_6,PUY_QNH,PUY_Wind_dir,PUY_Wind_dir_delta,PUY_Wind_dir_delta-3,PUY_Wind_dir_delta-6,PUY_Wind_dir_spread30,PUY_Wind_dir_spread60,PUY_Wind_max_EW,PUY_Wind_max_NS,PUY_Wind_max,PUY_Wind_max_delta_3,PUY_Wind_max_delta_6,PUY_Wind_speed_EW,PUY_Wind_speed_NS,PUY_Wind_speed,PUY_Wind_speed_delta_3,PUY_Wind_speed_delta_6,REH_Air_temp,REH_Hum,REH_Hum_delta_3,REH_Hum_delta_6,REH_QNH,REH_Wind_dir,REH_Wind_dir_delta,REH_Wind_dir_delta-3,REH_Wind_dir_delta-6,REH_Wind_dir_spread30,REH_Wind_dir_spread60,REH_Wind_max_EW,REH_Wind_max_NS,REH_Wind_max,REH_Wind_max_delta_3,REH_Wind_max_delta_6,REH_Wind_speed_EW,REH_Wind_speed_NS,REH_Wind_speed,REH_Wind_speed_delta_3,REH_Wind_speed_delta_6,SHA_Air_temp,SHA_Hum,SHA_Hum_delta_3,SHA_Hum_delta_6,SHA_QNH,SHA_Wind_dir,SHA_Wind_dir_delta,SHA_Wind_dir_delta-3,SHA_Wind_dir_delta-6,SHA_Wind_dir_spre

### Spreads recalculated 01.08.2019 after correction of angle differences

## Spreads in wind direction change
<ul><li>Wind_dir_spread30: RMSE of the changes of wind direction in past 30 mins. (already generated)</li>
<li>Wind_dir_spread60: RMSE of the changes of wind direction in past 60 mins. (new)</li>
</ul>


In [0]:
def calc_wind_dir_spread(vals):
    
    if len(vals) == 0:
        raise Exception('No values provided')
        
    sqrs = 0.0
    for val in vals:
        sqrs += val ** 2
    return np.sqrt(sqrs) / len(vals)


def calc_wind_dir_spreads(df1, cols_delta, spread, is_to_save=False, shift=1):
    
    df = df1.copy()
    for c in cols_delta:
        n = c.replace('delta', 'spread' + str(10*spread))
        print()
        text = f'Calculating delta spread in {n} from {c}'
        print(text)
        print(len(text) * '*')

        col_index = df.columns.get_loc(c)
        new_col_index = col_index + shift
        df.insert(new_col_index, n, np.NaN)
        
        for i, r in enumerate(df[c]):
            vals= []
            intv = spread
            if i < spread:
                intv = i
            display = []
            for j in range(spread):
                display.append(df[c].iloc[i-j])
                vals.append(df[c].iloc[i-j])
            val = calc_wind_dir_spread(vals)
            display.append(val)
            
            if i % 10000 == 0:
              print(display)
                
            df.iloc[i,new_col_index] = val
            # store every 10'000 calculations...never know
            if is_to_save:
                try:
                    if i%50000 == 0 and i > 0:
                        filename = 'angle_spread_temp.csv.zip'
                        dh.store_df_as_csv_compressed(df, )
                        print('Saved', filename)
                except:
                    print('Failure storing temporary calculations, calculations continue')
                
            if i%5000 == 1:
                print(val)
                
    return df

In [0]:
df_all_features = pd.read_csv(join(data_path, 'model_data_all_features_extended.csv.zip'), index_col=0)

In [0]:
np.sqrt(18**2 + 4**2 + 11**2)/3

7.156970184527963

In [10]:
#'model_data_all_features_extended.csv.zip'
df_all_features.head()

,datetime,KLO_Air_temp,KLO_Hum,KLO_Hum_delta_3,KLO_Hum_delta_6,KLO_QNH,KLO_Wind_dir,KLO_Wind_dir_delta,KLO_Wind_dir_delta-3,KLO_Wind_dir_delta-6,KLO_Wind_dir_spread30,KLO_Wind_dir_spread60,KLO_Wind_max_EW,KLO_Wind_max_NS,KLO_Wind_max,KLO_Wind_max_delta_3,KLO_Wind_max_delta_6,KLO_Wind_speed_EW,KLO_Wind_speed_NS,KLO_Wind_speed,KLO_Wind_speed_delta_3,KLO_Wind_speed_delta_6,BIZ_Air_temp,BIZ_Hum,BIZ_Hum_delta_3,BIZ_Hum_delta_6,BIZ_QNH,BIZ_Wind_dir,BIZ_Wind_dir_delta,BIZ_Wind_dir_delta-3,BIZ_Wind_dir_delta-6,BIZ_Wind_dir_spread30,BIZ_Wind_dir_spread60,BIZ_Wind_max_EW,BIZ_Wind_max_NS,BIZ_Wind_max,BIZ_Wind_max_delta_3,BIZ_Wind_max_delta_6,BIZ_Wind_speed_EW,BIZ_Wind_speed_NS,BIZ_Wind_speed,BIZ_Wind_speed_delta_3,BIZ_Wind_speed_delta_6,BUS_Air_temp,BUS_Hum,BUS_Hum_delta_3,BUS_Hum_delta_6,BUS_QNH,BUS_Wind_dir,BUS_Wind_dir_delta,BUS_Wind_dir_delta-3,BUS_Wind_dir_delta-6,BUS_Wind_dir_spread30,BUS_Wind_dir_spread60,BUS_Wind_max_EW,BUS_Wind_max_NS,BUS_Wind_max,BUS_Wind_max_delta_3,BUS_Wind_max_delta_6,BUS_Wind_speed_EW,BUS_Wind_speed_NS,BUS_Wind_speed,BUS_Wind_speed_delta_3,BUS_Wind_speed_delta_6,GUT_Air_temp,GUT_Hum,GUT_Hum_delta_3,GUT_Hum_delta_6,GUT_QNH,GUT_Wind_dir,GUT_Wind_dir_delta,GUT_Wind_dir_delta-3,GUT_Wind_dir_delta-6,GUT_Wind_dir_spread30,GUT_Wind_dir_spread60,GUT_Wind_max_EW,GUT_Wind_max_NS,GUT_Wind_max,GUT_Wind_max_delta_3,GUT_Wind_max_delta_6,GUT_Wind_speed_EW,GUT_Wind_speed_NS,GUT_Wind_speed,GUT_Wind_speed_delta_3,GUT_Wind_speed_delta_6,KOP_Air_temp,KOP_Hum,KOP_Hum_delta_3,KOP_Hum_delta_6,KOP_QNH,KOP_Wind_dir,KOP_Wind_dir_delta,KOP_Wind_dir_delta-3,KOP_Wind_dir_delta-6,KOP_Wind_dir_spread30,KOP_Wind_dir_spread60,KOP_Wind_max_EW,KOP_Wind_max_NS,KOP_Wind_max,KOP_Wind_max_delta_3,KOP_Wind_max_delta_6,KOP_Wind_speed_EW,KOP_Wind_speed_NS,KOP_Wind_speed,KOP_Wind_speed_delta_3,KOP_Wind_speed_delta_6,MOA_Air_temp,MOA_Hum,MOA_Hum_delta_3,MOA_Hum_delta_6,MOA_QNH,MOA_Wind_dir,MOA_Wind_dir_delta,MOA_Wind_dir_delta-3,MOA_Wind_dir_delta-6,MOA_Wind_dir_spread30,MOA_Wind_dir_spread60,MOA_Wind_max_EW,MOA_Wind_max_NS,MOA_Wind_max,MOA_Wind_max_delta_3,MOA_Wind_max_delta_6,MOA_Wind_speed_EW,MOA_Wind_speed_NS,MOA_Wind_speed,MOA_Wind_speed_delta_3,MOA_Wind_speed_delta_6,MUB_Air_temp,MUB_Hum,MUB_Hum_delta_3,MUB_Hum_delta_6,MUB_QNH,MUB_Wind_dir,MUB_Wind_dir_delta,MUB_Wind_dir_delta-3,MUB_Wind_dir_delta-6,MUB_Wind_dir_spread30,MUB_Wind_dir_spread60,MUB_Wind_max_EW,MUB_Wind_max_NS,MUB_Wind_max,MUB_Wind_max_delta_3,MUB_Wind_max_delta_6,MUB_Wind_speed_EW,MUB_Wind_speed_NS,MUB_Wind_speed,MUB_Wind_speed_delta_3,MUB_Wind_speed_delta_6,PAY_Air_temp,PAY_Hum,PAY_Hum_delta_3,PAY_Hum_delta_6,PAY_QNH,PAY_Wind_dir,PAY_Wind_dir_delta,PAY_Wind_dir_delta-3,PAY_Wind_dir_delta-6,PAY_Wind_dir_spread30,PAY_Wind_dir_spread60,PAY_Wind_max_EW,PAY_Wind_max_NS,PAY_Wind_max,PAY_Wind_max_delta_3,PAY_Wind_max_delta_6,PAY_Wind_speed_EW,PAY_Wind_speed_NS,PAY_Wind_speed,PAY_Wind_speed_delta_3,PAY_Wind_speed_delta_6,PUY_Air_temp,PUY_Hum,PUY_Hum_delta_3,PUY_Hum_delta_6,PUY_QNH,PUY_Wind_dir,PUY_Wind_dir_delta,PUY_Wind_dir_delta-3,PUY_Wind_dir_delta-6,PUY_Wind_dir_spread30,PUY_Wind_dir_spread60,PUY_Wind_max_EW,PUY_Wind_max_NS,PUY_Wind_max,PUY_Wind_max_delta_3,PUY_Wind_max_delta_6,PUY_Wind_speed_EW,PUY_Wind_speed_NS,PUY_Wind_speed,PUY_Wind_speed_delta_3,PUY_Wind_speed_delta_6,REH_Air_temp,REH_Hum,REH_Hum_delta_3,REH_Hum_delta_6,REH_QNH,REH_Wind_dir,REH_Wind_dir_delta,REH_Wind_dir_delta-3,REH_Wind_dir_delta-6,REH_Wind_dir_spread30,REH_Wind_dir_spread60,REH_Wind_max_EW,REH_Wind_max_NS,REH_Wind_max,REH_Wind_max_delta_3,REH_Wind_max_delta_6,REH_Wind_speed_EW,REH_Wind_speed_NS,REH_Wind_speed,REH_Wind_speed_delta_3,REH_Wind_speed_delta_6,SHA_Air_temp,SHA_Hum,SHA_Hum_delta_3,SHA_Hum_delta_6,SHA_QNH,SHA_Wind_dir,SHA_Wind_dir_delta,SHA_Wind_dir_delta-3,SHA_Wind_dir_delta-6,SHA_Wind_dir_spread30,SHA_Wind_dir_spread60,SHA_Wind_max_EW,SHA_Wind_max_NS,SHA_Wind_max,SHA_Wind_max_delta_3,SHA_Wind_max_delta_6,SHA_Wind_speed_EW,SHA_Wind_speed_NS,SHA_Wind_speed,SHA_Wind_speed_delta_3,SHA_Wind_speed_delta

In [11]:
cols_delta = [c for c in df_all_features.columns if re.search('^[A-Z]{3}_Wind_dir_.*delta$', c)]
cols_delta

['KLO_Wind_dir_delta',
 'BIZ_Wind_dir_delta',
 'BUS_Wind_dir_delta',
 'GUT_Wind_dir_delta',
 'KOP_Wind_dir_delta',
 'MOA_Wind_dir_delta',
 'MUB_Wind_dir_delta',
 'PAY_Wind_dir_delta',
 'PUY_Wind_dir_delta',
 'REH_Wind_dir_delta',
 'SHA_Wind_dir_delta',
 'SMA_Wind_dir_delta',
 'TAE_Wind_dir_delta',
 'VAD_Wind_dir_delta',
 'WAE_Wind_dir_delta',
 'WYN_Wind_dir_delta']

In [0]:
cols_spread = [c for c in df_all_features.columns if re.search('spread60', c)]
mid = int(len(cols_spread) / 2)
c1 = cols_spread[0:mid]
c2 = cols_spread[mid:len(cols_spread)]
df = calc_wind_dir_spreads(df_all_features.drop(columns=c1), cols_delta[0:mid], 6, is_to_save=False, shift=4)
print(c1, c2)
print(cols_delta)


Calculating delta spread in KLO_Wind_dir_spread60 from KLO_Wind_dir_delta
*************************************************************************
[nan, 3.0, 45.0, 37.0, 154.0, 158.0, nan]
nan
10.318537794776072
[4.0, 11.0, -16.0, -10.0, 16.0, -20.0, 5.649483752226097]
4.708148963941845
2.8771127502720115
[-18.0, 3.0, -5.0, -19.0, -5.0, 6.0, 4.654746681256314]
4.901813723284247
8.709828420302612
[52.0, -5.0, -12.0, 0.0, 8.0, 23.0, 9.812124019690017]
10.244239573752873
14.807092894960848
[4.0, -9.0, -16.0, -7.0, -22.0, -1.0, 4.963757537278477]
4.960958688712405
32.85320887422313
[5.0, 1.0, -11.0, -15.0, -19.0, 24.0, 6.030017505041847]
6.141841924229426
5.515130702591432
[-4.0, 2.0, 11.0, 0.0, -11.0, -2.0, 2.7182510717166815]
2.7182510717166815
27.705795623138332
[-7.0, 27.0, -29.0, -30.0, -8.0, 48.0, 11.651180197731044]
8.666666666666666
18.715709384839727
[-6.0, 10.0, -19.0, 15.0, -11.0, -9.0, 5.066228051190222]
5.607534613753574
20.726392192886184
[-31.0, -26.0, 49.0, 34.0, -20.0, -

In [0]:
df.head(10)

,datetime,KLO_Air_temp,KLO_Hum,KLO_Hum_delta_3,KLO_Hum_delta_6,KLO_QNH,KLO_Wind_dir,KLO_Wind_dir_delta,KLO_Wind_dir_delta-3,KLO_Wind_dir_delta-6,KLO_Wind_dir_spread30,KLO_Wind_dir_spread60,KLO_Wind_max_EW,KLO_Wind_max_NS,KLO_Wind_max,KLO_Wind_max_delta_3,KLO_Wind_max_delta_6,KLO_Wind_speed_EW,KLO_Wind_speed_NS,KLO_Wind_speed,KLO_Wind_speed_delta_3,KLO_Wind_speed_delta_6,BIZ_Air_temp,BIZ_Hum,BIZ_Hum_delta_3,BIZ_Hum_delta_6,BIZ_QNH,BIZ_Wind_dir,BIZ_Wind_dir_delta,BIZ_Wind_dir_delta-3,BIZ_Wind_dir_delta-6,BIZ_Wind_dir_spread30,BIZ_Wind_dir_spread60,BIZ_Wind_max_EW,BIZ_Wind_max_NS,BIZ_Wind_max,BIZ_Wind_max_delta_3,BIZ_Wind_max_delta_6,BIZ_Wind_speed_EW,BIZ_Wind_speed_NS,BIZ_Wind_speed,BIZ_Wind_speed_delta_3,BIZ_Wind_speed_delta_6,BUS_Air_temp,BUS_Hum,BUS_Hum_delta_3,BUS_Hum_delta_6,BUS_QNH,BUS_Wind_dir,BUS_Wind_dir_delta,BUS_Wind_dir_delta-3,BUS_Wind_dir_delta-6,BUS_Wind_dir_spread30,BUS_Wind_dir_spread60,BUS_Wind_max_EW,BUS_Wind_max_NS,BUS_Wind_max,BUS_Wind_max_delta_3,BUS_Wind_max_delta_6,BUS_Wind_speed_EW,BUS_Wind_speed_NS,BUS_Wind_speed,BUS_Wind_speed_delta_3,BUS_Wind_speed_delta_6,GUT_Air_temp,GUT_Hum,GUT_Hum_delta_3,GUT_Hum_delta_6,GUT_QNH,GUT_Wind_dir,GUT_Wind_dir_delta,GUT_Wind_dir_delta-3,GUT_Wind_dir_delta-6,GUT_Wind_dir_spread30,GUT_Wind_dir_spread60,GUT_Wind_max_EW,GUT_Wind_max_NS,GUT_Wind_max,GUT_Wind_max_delta_3,GUT_Wind_max_delta_6,GUT_Wind_speed_EW,GUT_Wind_speed_NS,GUT_Wind_speed,GUT_Wind_speed_delta_3,GUT_Wind_speed_delta_6,KOP_Air_temp,KOP_Hum,KOP_Hum_delta_3,KOP_Hum_delta_6,KOP_QNH,KOP_Wind_dir,KOP_Wind_dir_delta,KOP_Wind_dir_delta-3,KOP_Wind_dir_delta-6,KOP_Wind_dir_spread30,KOP_Wind_dir_spread60,KOP_Wind_max_EW,KOP_Wind_max_NS,KOP_Wind_max,KOP_Wind_max_delta_3,KOP_Wind_max_delta_6,KOP_Wind_speed_EW,KOP_Wind_speed_NS,KOP_Wind_speed,KOP_Wind_speed_delta_3,KOP_Wind_speed_delta_6,MOA_Air_temp,MOA_Hum,MOA_Hum_delta_3,MOA_Hum_delta_6,MOA_QNH,MOA_Wind_dir,MOA_Wind_dir_delta,MOA_Wind_dir_delta-3,MOA_Wind_dir_delta-6,MOA_Wind_dir_spread30,MOA_Wind_dir_spread60,MOA_Wind_max_EW,MOA_Wind_max_NS,MOA_Wind_max,MOA_Wind_max_delta_3,MOA_Wind_max_delta_6,MOA_Wind_speed_EW,MOA_Wind_speed_NS,MOA_Wind_speed,MOA_Wind_speed_delta_3,MOA_Wind_speed_delta_6,MUB_Air_temp,MUB_Hum,MUB_Hum_delta_3,MUB_Hum_delta_6,MUB_QNH,MUB_Wind_dir,MUB_Wind_dir_delta,MUB_Wind_dir_delta-3,MUB_Wind_dir_delta-6,MUB_Wind_dir_spread30,MUB_Wind_dir_spread60,MUB_Wind_max_EW,MUB_Wind_max_NS,MUB_Wind_max,MUB_Wind_max_delta_3,MUB_Wind_max_delta_6,MUB_Wind_speed_EW,MUB_Wind_speed_NS,MUB_Wind_speed,MUB_Wind_speed_delta_3,MUB_Wind_speed_delta_6,PAY_Air_temp,PAY_Hum,PAY_Hum_delta_3,PAY_Hum_delta_6,PAY_QNH,PAY_Wind_dir,PAY_Wind_dir_delta,PAY_Wind_dir_delta-3,PAY_Wind_dir_delta-6,PAY_Wind_dir_spread30,PAY_Wind_dir_spread60,PAY_Wind_max_EW,PAY_Wind_max_NS,PAY_Wind_max,PAY_Wind_max_delta_3,PAY_Wind_max_delta_6,PAY_Wind_speed_EW,PAY_Wind_speed_NS,PAY_Wind_speed,PAY_Wind_speed_delta_3,PAY_Wind_speed_delta_6,PUY_Air_temp,PUY_Hum,PUY_Hum_delta_3,PUY_Hum_delta_6,PUY_QNH,PUY_Wind_dir,PUY_Wind_dir_delta,PUY_Wind_dir_delta-3,PUY_Wind_dir_delta-6,PUY_Wind_dir_spread30,PUY_Wind_dir_spread60,PUY_Wind_max_EW,PUY_Wind_max_NS,PUY_Wind_max,PUY_Wind_max_delta_3,PUY_Wind_max_delta_6,PUY_Wind_speed_EW,PUY_Wind_speed_NS,PUY_Wind_speed,PUY_Wind_speed_delta_3,PUY_Wind_speed_delta_6,REH_Air_temp,REH_Hum,REH_Hum_delta_3,REH_Hum_delta_6,REH_QNH,REH_Wind_dir,REH_Wind_dir_delta,REH_Wind_dir_delta-3,REH_Wind_dir_delta-6,REH_Wind_dir_spread30,REH_Wind_dir_spread60,REH_Wind_max_EW,REH_Wind_max_NS,REH_Wind_max,REH_Wind_max_delta_3,REH_Wind_max_delta_6,REH_Wind_speed_EW,REH_Wind_speed_NS,REH_Wind_speed,REH_Wind_speed_delta_3,REH_Wind_speed_delta_6,SHA_Air_temp,SHA_Hum,SHA_Hum_delta_3,SHA_Hum_delta_6,SHA_QNH,SHA_Wind_dir,SHA_Wind_dir_delta,SHA_Wind_dir_delta-3,SHA_Wind_dir_delta-6,SHA_Wind_dir_spread30,SHA_Wind_dir_spread60,SHA_Wind_max_EW,SHA_Wind_max_NS,SHA_Wind_max,SHA_Wind_max_delta_3,SHA_Wind_max_delta_6,SHA_Wind_speed_EW,SHA_Wind_speed_NS,SHA_Wind_speed,SHA_Wind_speed_delta_3,SHA_Wind_speed_delta

In [12]:
df = calc_wind_dir_spreads(df_all_features.drop(columns=c2), cols_delta[mid:], 6, is_to_save=False, shift=4)
df.to_csv(join(data_path, 'model_data_all_features_extended.csv'))

NameError: ignored

In [0]:
cols_spread = [c for c in df_all_features.columns if re.search('spread30', c)]
mid = int(len(cols_spread) / 2)
c1 = cols_spread[0:mid]
c2 = cols_spread[mid:len(cols_spread)]
df = calc_wind_dir_spreads(df_all_features.drop(columns=c1), cols_delta[0:mid], steps_30_mins, is_to_save=False, shift=3)
print(c1, c2)
print(cols_delta)
df.to_csv(join(data_path, 'model_data_all_features_extended.csv'))


Calculating delta spread in KLO_Wind_dir_spread30 from KLO_Wind_dir_delta
*************************************************************************
[nan, 3.0, 45.0, nan]
nan
16.609903337734657
[4.0, 11.0, -16.0, 6.60807586719967]
4.546060565661952
1.9436506316151003
[-18.0, 3.0, -5.0, 6.306962642808167]
7.102425250887506
14.884742374510738
[52.0, -5.0, -12.0, 17.866791044343195]
19.91649232838621
8.602325267042627
[4.0, -9.0, -16.0, 6.262764742685312]
3.2829526005987013
30.973106614179557
[5.0, 1.0, -11.0, 4.041451884327381]
8.504900548115382
4.384315479321969
[-4.0, 2.0, 11.0, 3.958114029012639]
1.6329931618554518
53.19774431308155
[-7.0, 27.0, -29.0, 13.412266359153806]
9.994442900376633
3.6055512754639896
[-6.0, 10.0, -19.0, 7.431165603202651]
6.871842709362768
12.427567921538165
[-31.0, -26.0, 49.0, 21.18175315375634]
13.589211407093005
9.285592184789413
[3.0, 10.0, -10.0, 4.818944098266987]
6.936217348894938
4.9216076867444665
[22.0, 13.0, -16.0, 10.04987562112089]
10.81665382639

In [9]:
cols_spread = [c for c in df_all_features.columns if re.search('spread30', c)]
mid = int(len(cols_spread) / 2)
c1 = cols_spread[0:mid]
c2 = cols_spread[mid:len(cols_spread)]
df = calc_wind_dir_spreads(df_all_features.drop(columns=c2), cols_delta[mid:], steps_30_mins, is_to_save=False, shift=3)
print(c1, c2)
print(cols_delta)
df.to_csv(join(data_path, 'model_data_all_features_extended.csv'))


Calculating delta spread in PUY_Wind_dir_spread30 from PUY_Wind_dir_delta
*************************************************************************
[nan, -3.0, -5.0, nan]
nan
10.466878978738388
[-5.0, -7.0, 1.0, 2.886751345948129]
8.812869377601524
2.4267032964268394
[36.0, 38.0, 1.0, 17.451520150277897]
18.6785676348292
8.171767114754175
[-47.0, 45.0, 36.0, 24.787989923437607]
27.15183644290423
4.521553322083512
[-1.0, 22.0, 11.0, 8.205689083394114]
7.52772652709081
17.30767331432956
[0.0, -2.0, 16.0, 5.374838498865699]
3.3993463423951895
1.2018504251546631
[0.0, 0.0, 0.0, 0.0]
0.0
12.565384549980509
[-17.0, 125.0, 0.0, 42.05023450852828]
55.355417279813025
33.441491194821644
[14.0, -43.0, 28.0, 17.729448195962934]
15.253414918196734
0.0
[-4.0, 1.0, -5.0, 2.160246899469287]
1.9148542155126762
14.779866184930242
[101.0, -123.0, -5.0, 53.077511454685045]
58.789643834796465
26.14702022538451
[13.0, -14.0, -10.0, 7.187952884282608]
23.223551264467144
30.94798503582709
[-5.0, -12.0, 19.0,

In [0]:
cols_spread = [c for c in df_all_features.columns if re.search('spread60', c)]
mid = int(len(cols_spread) / 2)
c1 = cols_spread[0:mid]
c2 = cols_spread[mid:len(cols_spread)]

df = calc_wind_dir_spreads(df_all_features.drop(columns=c1), cols_delta[:mid], steps_1_hour, is_to_save=False, shift=4)
print(c1, c2)
print(cols_delta)
df.to_csv(join(data_path, 'model_data_all_features_extended.csv'))


Calculating delta spread in KLO_Wind_dir_spread60 from KLO_Wind_dir_delta
*************************************************************************
[nan, 3.0, 45.0, 37.0, 154.0, 158.0, nan]
nan
10.318537794776072
[4.0, 11.0, -16.0, -10.0, 16.0, -20.0, 5.649483752226097]
4.708148963941845


In [0]:
#df_all_features = pd.read_csv(join(data_path, 'model_data_all_features_extended.csv.zip'), index_col=0)
cols_spread = [c for c in df_all_features.columns if re.search('spread30', c)]
mid = int(len(cols_spread) / 2)
c1 = cols_spread[0:mid]
c2 = cols_spread[mid:len(cols_spread)]
df = calc_wind_dir_spreads(df.drop(columns=c1), cols_delta[0:mid], steps_30_mins, is_to_save=False, shift=3)
print(c1, c2)
print(cols_delta)
df.to_csv(join(data_path, 'model_data_all_features_extended.csv'))

### Check where corrections made

In [0]:
# custom scripts
sys.path.append('../gdrive/My Drive/Colab Notebooks')
import data_handler as dh
import feature_handler as fh
import data_processor as dp
import constants as const
import work_utils as wu
import plot_handler as ph
import xgb_handler as xh

In [0]:
cols_all_spread = [c for c in df_all_features.columns if re.search('spread', c)]
df_original = df_all_features[['datetime'] + cols_all_spread].copy()
df_original.head()

,datetime,KLO_Wind_dir_spread30,KLO_Wind_dir_spread60,BIZ_Wind_dir_spread30,BIZ_Wind_dir_spread60,BUS_Wind_dir_spread30,BUS_Wind_dir_spread60,GUT_Wind_dir_spread30,GUT_Wind_dir_spread60,KOP_Wind_dir_spread30,KOP_Wind_dir_spread60,MOA_Wind_dir_spread30,MOA_Wind_dir_spread60,MUB_Wind_dir_spread30,MUB_Wind_dir_spread60,PAY_Wind_dir_spread30,PAY_Wind_dir_spread60,PUY_Wind_dir_spread30,PUY_Wind_dir_spread60,REH_Wind_dir_spread30,REH_Wind_dir_spread60,SHA_Wind_dir_spread30,SHA_Wind_dir_spread60,SMA_Wind_dir_spread30,SMA_Wind_dir_spread60,TAE_Wind_dir_spread30,TAE_Wind_dir_spread60,VAD_Wind_dir_spread30,VAD_Wind_dir_spread60,WAE_Wind_dir_spread30,WAE_Wind_dir_spread60,WYN_Wind_dir_spread30,WYN_Wind_dir_spread60
0,2014-01-01 00:00:00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2014-01-01 00:10:00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,2014-01-01 00:20:00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,2014-01-01 00:30:00,7.16,nan,15.43,nan,17.14,nan,12.90,nan,25.80,nan,12.02,nan,2.16,nan,24.18,nan,111.70,nan,9.41,nan,1.00,nan,3.82,nan,104.09,nan,5.91,nan,8.12,nan,8.44,nan
4,2014-01-01 00:40:00,6.29,nan,14.22,nan,17.15,nan,21.48,nan,23.45,nan,6.16,nan,2.62,nan,40.81,nan,57.39,nan,10.73,nan,11.69,nan,6.55,nan,38.62,nan,7.13,nan,7.55,nan,8.54,nan


In [0]:
re_names = {}
for c in cols_all_spread:
  re_names[c] = c + '_orig'
df_original.rename(columns=re_names, inplace=True)
df_original.head()

,datetime,KLO_Wind_dir_spread30_orig,KLO_Wind_dir_spread60_orig,BIZ_Wind_dir_spread30_orig,BIZ_Wind_dir_spread60_orig,BUS_Wind_dir_spread30_orig,BUS_Wind_dir_spread60_orig,GUT_Wind_dir_spread30_orig,GUT_Wind_dir_spread60_orig,KOP_Wind_dir_spread30_orig,KOP_Wind_dir_spread60_orig,MOA_Wind_dir_spread30_orig,MOA_Wind_dir_spread60_orig,MUB_Wind_dir_spread30_orig,MUB_Wind_dir_spread60_orig,PAY_Wind_dir_spread30_orig,PAY_Wind_dir_spread60_orig,PUY_Wind_dir_spread30_orig,PUY_Wind_dir_spread60_orig,REH_Wind_dir_spread30_orig,REH_Wind_dir_spread60_orig,SHA_Wind_dir_spread30_orig,SHA_Wind_dir_spread60_orig,SMA_Wind_dir_spread30_orig,SMA_Wind_dir_spread60_orig,TAE_Wind_dir_spread30_orig,TAE_Wind_dir_spread60_orig,VAD_Wind_dir_spread30_orig,VAD_Wind_dir_spread60_orig,WAE_Wind_dir_spread30_orig,WAE_Wind_dir_spread60_orig,WYN_Wind_dir_spread30_orig,WYN_Wind_dir_spread60_orig
0,2014-01-01 00:00:00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2014-01-01 00:10:00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,2014-01-01 00:20:00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,2014-01-01 00:30:00,7.16,nan,15.43,nan,17.14,nan,12.90,nan,25.80,nan,12.02,nan,2.16,nan,24.18,nan,111.70,nan,9.41,nan,1.00,nan,3.82,nan,104.09,nan,5.91,nan,8.12,nan,8.44,nan
4,2014-01-01 00:40:00,6.29,nan,14.22,nan,17.15,nan,21.48,nan,23.45,nan,6.16,nan,2.62,nan,40.81,nan,57.39,nan,10.73,nan,11.69,nan,6.55,nan,38.62,nan,7.13,nan,7.55,nan,8.54,nan


In [0]:
df_check = dp.left_join_datasets(df[['datetime'] + cols_all_spread], df_original, on='datetime')

for c in cols_all_spread:
  drop_col = c + '_drop' 
  df_check[drop_col] = df_check[c] == df_check[c + '_orig']
  
last = len(df_check.columns)

In [0]:
#df_check['checkpoint'] = df_check.iloc[:,last]
df_check.head(20)

,datetime,KLO_Wind_dir_spread30,KLO_Wind_dir_spread60,BIZ_Wind_dir_spread30,BIZ_Wind_dir_spread60,BUS_Wind_dir_spread30,BUS_Wind_dir_spread60,GUT_Wind_dir_spread30,GUT_Wind_dir_spread60,KOP_Wind_dir_spread30,KOP_Wind_dir_spread60,MOA_Wind_dir_spread30,MOA_Wind_dir_spread60,MUB_Wind_dir_spread30,MUB_Wind_dir_spread60,PAY_Wind_dir_spread30,PAY_Wind_dir_spread60,PUY_Wind_dir_spread30,PUY_Wind_dir_spread60,REH_Wind_dir_spread30,REH_Wind_dir_spread60,SHA_Wind_dir_spread30,SHA_Wind_dir_spread60,SMA_Wind_dir_spread30,SMA_Wind_dir_spread60,TAE_Wind_dir_spread30,TAE_Wind_dir_spread60,VAD_Wind_dir_spread30,VAD_Wind_dir_spread60,WAE_Wind_dir_spread30,WAE_Wind_dir_spread60,WYN_Wind_dir_spread30,WYN_Wind_dir_spread60,KLO_Wind_dir_spread30_orig,KLO_Wind_dir_spread60_orig,BIZ_Wind_dir_spread30_orig,BIZ_Wind_dir_spread60_orig,BUS_Wind_dir_spread30_orig,BUS_Wind_dir_spread60_orig,GUT_Wind_dir_spread30_orig,GUT_Wind_dir_spread60_orig,KOP_Wind_dir_spread30_orig,KOP_Wind_dir_spread60_orig,MOA_Wind_dir_spread30_orig,MOA_Wind_dir_spread60_orig,MUB_Wind_dir_spread30_orig,MUB_Wind_dir_spread60_orig,PAY_Wind_dir_spread30_orig,PAY_Wind_dir_spread60_orig,PUY_Wind_dir_spread30_orig,PUY_Wind_dir_spread60_orig,REH_Wind_dir_spread30_orig,REH_Wind_dir_spread60_orig,SHA_Wind_dir_spread30_orig,SHA_Wind_dir_spread60_orig,SMA_Wind_dir_spread30_orig,SMA_Wind_dir_spread60_orig,TAE_Wind_dir_spread30_orig,TAE_Wind_dir_spread60_orig,VAD_Wind_dir_spread30_orig,VAD_Wind_dir_spread60_orig,WAE_Wind_dir_spread30_orig,WAE_Wind_dir_spread60_orig,WYN_Wind_dir_spread30_orig,WYN_Wind_dir_spread60_orig,KLO_Wind_dir_spread30_drop,KLO_Wind_dir_spread60_drop,BIZ_Wind_dir_spread30_drop,BIZ_Wind_dir_spread60_drop,BUS_Wind_dir_spread30_drop,BUS_Wind_dir_spread60_drop,GUT_Wind_dir_spread30_drop,GUT_Wind_dir_spread60_drop,KOP_Wind_dir_spread30_drop,KOP_Wind_dir_spread60_drop,MOA_Wind_dir_spread30_drop,MOA_Wind_dir_spread60_drop,MUB_Wind_dir_spread30_drop,MUB_Wind_dir_spread60_drop,PAY_Wind_dir_spread30_drop,PAY_Wind_dir_spread60_drop,PUY_Wind_dir_spread30_drop,PUY_Wind_dir_spread60_drop,REH_Wind_dir_spread30_drop,REH_Wind_dir_spread60_drop,SHA_Wind_dir_spread30_drop,SHA_Wind_dir_spread60_drop,SMA_Wind_dir_spread30_drop,SMA_Wind_dir_spread60_drop,TAE_Wind_dir_spread30_drop,TAE_Wind_dir_spread60_drop,VAD_Wind_dir_spread30_drop,VAD_Wind_dir_spread60_drop,WAE_Wind_dir_spread30_drop,WAE_Wind_dir_spread60_drop,WYN_Wind_dir_spread30_drop,WYN_Wind_dir_spread60_drop
0,2014-01-01 00:00:00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,2014-01-01 00:10:00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,2014-01-01 00:20:00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,2014-01-01 00:30:00,7.16,nan,15.43,nan,17.14,nan,12.90,nan,25.80,nan,12.02,nan,2.16,nan,24.18,nan,111.70,nan,9.41,nan,1.00,nan,3.82,nan,104.09,nan

In [0]:
#KLO_Wind_dir_spread30	KLO_Wind_dir_spread60

print(df_check.iloc[8,df_check.columns.get_loc('KLO_Wind_dir_spread60')], df_check.iloc[8,df_check.columns.get_loc('KLO_Wind_dir_spread60_orig')], df_check.iloc[8,df_check.columns.get_loc('KLO_Wind_dir_spread60_drop')])

6.637435917246626 6.637435917246625 False


In [0]:
# random cross-check of spread values
np.sqrt(15.0**2 + 2.0**2+ 4.0**2+ 18.0**2+ 4.0**2) /6

4.031128874149275

In [0]:
df_all_features = pd.read_csv(join(data_path, 'model_data_all_features_extended.csv.zip'), index_col=0)
df_all_features.head(20)

,datetime,KLO_Air_temp,KLO_Hum,KLO_Hum_delta_3,KLO_Hum_delta_6,KLO_QNH,KLO_Wind_dir,KLO_Wind_dir_delta,KLO_Wind_dir_delta-3,KLO_Wind_dir_delta-6,KLO_Wind_dir_spread30,KLO_Wind_dir_spread60,KLO_Wind_max_EW,KLO_Wind_max_NS,KLO_Wind_max,KLO_Wind_max_delta_3,KLO_Wind_max_delta_6,KLO_Wind_speed_EW,KLO_Wind_speed_NS,KLO_Wind_speed,KLO_Wind_speed_delta_3,KLO_Wind_speed_delta_6,BIZ_Air_temp,BIZ_Hum,BIZ_Hum_delta_3,BIZ_Hum_delta_6,BIZ_QNH,BIZ_Wind_dir,BIZ_Wind_dir_delta,BIZ_Wind_dir_delta-3,BIZ_Wind_dir_delta-6,BIZ_Wind_dir_spread30,BIZ_Wind_dir_spread60,BIZ_Wind_max_EW,BIZ_Wind_max_NS,BIZ_Wind_max,BIZ_Wind_max_delta_3,BIZ_Wind_max_delta_6,BIZ_Wind_speed_EW,BIZ_Wind_speed_NS,BIZ_Wind_speed,BIZ_Wind_speed_delta_3,BIZ_Wind_speed_delta_6,BUS_Air_temp,BUS_Hum,BUS_Hum_delta_3,BUS_Hum_delta_6,BUS_QNH,BUS_Wind_dir,BUS_Wind_dir_delta,BUS_Wind_dir_delta-3,BUS_Wind_dir_delta-6,BUS_Wind_dir_spread30,BUS_Wind_dir_spread60,BUS_Wind_max_EW,BUS_Wind_max_NS,BUS_Wind_max,BUS_Wind_max_delta_3,BUS_Wind_max_delta_6,BUS_Wind_speed_EW,BUS_Wind_speed_NS,BUS_Wind_speed,BUS_Wind_speed_delta_3,BUS_Wind_speed_delta_6,GUT_Air_temp,GUT_Hum,GUT_Hum_delta_3,GUT_Hum_delta_6,GUT_QNH,GUT_Wind_dir,GUT_Wind_dir_delta,GUT_Wind_dir_delta-3,GUT_Wind_dir_delta-6,GUT_Wind_dir_spread30,GUT_Wind_dir_spread60,GUT_Wind_max_EW,GUT_Wind_max_NS,GUT_Wind_max,GUT_Wind_max_delta_3,GUT_Wind_max_delta_6,GUT_Wind_speed_EW,GUT_Wind_speed_NS,GUT_Wind_speed,GUT_Wind_speed_delta_3,GUT_Wind_speed_delta_6,KOP_Air_temp,KOP_Hum,KOP_Hum_delta_3,KOP_Hum_delta_6,KOP_QNH,KOP_Wind_dir,KOP_Wind_dir_delta,KOP_Wind_dir_delta-3,KOP_Wind_dir_delta-6,KOP_Wind_dir_spread30,KOP_Wind_dir_spread60,KOP_Wind_max_EW,KOP_Wind_max_NS,KOP_Wind_max,KOP_Wind_max_delta_3,KOP_Wind_max_delta_6,KOP_Wind_speed_EW,KOP_Wind_speed_NS,KOP_Wind_speed,KOP_Wind_speed_delta_3,KOP_Wind_speed_delta_6,MOA_Air_temp,MOA_Hum,MOA_Hum_delta_3,MOA_Hum_delta_6,MOA_QNH,MOA_Wind_dir,MOA_Wind_dir_delta,MOA_Wind_dir_delta-3,MOA_Wind_dir_delta-6,MOA_Wind_dir_spread30,MOA_Wind_dir_spread60,MOA_Wind_max_EW,MOA_Wind_max_NS,MOA_Wind_max,MOA_Wind_max_delta_3,MOA_Wind_max_delta_6,MOA_Wind_speed_EW,MOA_Wind_speed_NS,MOA_Wind_speed,MOA_Wind_speed_delta_3,MOA_Wind_speed_delta_6,MUB_Air_temp,MUB_Hum,MUB_Hum_delta_3,MUB_Hum_delta_6,MUB_QNH,MUB_Wind_dir,MUB_Wind_dir_delta,MUB_Wind_dir_delta-3,MUB_Wind_dir_delta-6,MUB_Wind_dir_spread30,MUB_Wind_dir_spread60,MUB_Wind_max_EW,MUB_Wind_max_NS,MUB_Wind_max,MUB_Wind_max_delta_3,MUB_Wind_max_delta_6,MUB_Wind_speed_EW,MUB_Wind_speed_NS,MUB_Wind_speed,MUB_Wind_speed_delta_3,MUB_Wind_speed_delta_6,PAY_Air_temp,PAY_Hum,PAY_Hum_delta_3,PAY_Hum_delta_6,PAY_QNH,PAY_Wind_dir,PAY_Wind_dir_delta,PAY_Wind_dir_delta-3,PAY_Wind_dir_delta-6,PAY_Wind_dir_spread30,PAY_Wind_dir_spread60,PAY_Wind_max_EW,PAY_Wind_max_NS,PAY_Wind_max,PAY_Wind_max_delta_3,PAY_Wind_max_delta_6,PAY_Wind_speed_EW,PAY_Wind_speed_NS,PAY_Wind_speed,PAY_Wind_speed_delta_3,PAY_Wind_speed_delta_6,PUY_Air_temp,PUY_Hum,PUY_Hum_delta_3,PUY_Hum_delta_6,PUY_QNH,PUY_Wind_dir,PUY_Wind_dir_delta,PUY_Wind_dir_delta-3,PUY_Wind_dir_delta-6,PUY_Wind_dir_spread30,PUY_Wind_dir_spread60,PUY_Wind_max_EW,PUY_Wind_max_NS,PUY_Wind_max,PUY_Wind_max_delta_3,PUY_Wind_max_delta_6,PUY_Wind_speed_EW,PUY_Wind_speed_NS,PUY_Wind_speed,PUY_Wind_speed_delta_3,PUY_Wind_speed_delta_6,REH_Air_temp,REH_Hum,REH_Hum_delta_3,REH_Hum_delta_6,REH_QNH,REH_Wind_dir,REH_Wind_dir_delta,REH_Wind_dir_delta-3,REH_Wind_dir_delta-6,REH_Wind_dir_spread30,REH_Wind_dir_spread60,REH_Wind_max_EW,REH_Wind_max_NS,REH_Wind_max,REH_Wind_max_delta_3,REH_Wind_max_delta_6,REH_Wind_speed_EW,REH_Wind_speed_NS,REH_Wind_speed,REH_Wind_speed_delta_3,REH_Wind_speed_delta_6,SHA_Air_temp,SHA_Hum,SHA_Hum_delta_3,SHA_Hum_delta_6,SHA_QNH,SHA_Wind_dir,SHA_Wind_dir_delta,SHA_Wind_dir_delta-3,SHA_Wind_dir_delta-6,SHA_Wind_dir_spread30,SHA_Wind_dir_spread60,SHA_Wind_max_EW,SHA_Wind_max_NS,SHA_Wind_max,SHA_Wind_max_delta_3,SHA_Wind_max_delta_6,SHA_Wind_speed_EW,SHA_Wind_speed_NS,SHA_Wind_speed,SHA_Wind_speed_delta_3,SHA_Wind_speed_delta

In [0]:
df_class = pd.read_csv(join(data_path, 'bise_classifications_extended.csv.zip'), index_col=0)
df_class.head()

,datetime,original_index,year,month,bise_event,rule3_episode,rule4_episode,rule3_duration,rule4_duration,rule3_label,rule4_label,manual_bise_x,manual_bise_y,manual_duration_class,rule4_episode_1_0,rule4_duration_class
0,2014-01-01 00:00:00,0,2014,1,0,0,0,0.00,0.00,0,0,0,0,0,0,0
1,2014-01-01 00:10:00,1,2014,1,0,0,0,0.00,0.00,0,0,0,0,0,0,0
2,2014-01-01 00:20:00,2,2014,1,0,0,0,0.00,0.00,0,0,0,0,0,0,0
3,2014-01-01 00:30:00,3,2014,1,0,0,0,0.00,0.00,0,0,0,0,0,0,0
4,2014-01-01 00:40:00,4,2014,1,0,0,0,0.00,0.00,0,0,0,0,0,0,0


In [0]:
df_merge = pd.merge(df_class, df_all_features, on='datetime')
df_merge.head()

,datetime,original_index,year,month,bise_event,rule3_episode,rule4_episode,rule3_duration,rule4_duration,rule3_label,rule4_label,manual_bise_x,manual_bise_y,manual_duration_class,rule4_episode_1_0,rule4_duration_class,KLO_Air_temp,KLO_Hum,KLO_Hum_delta_3,KLO_Hum_delta_6,KLO_QNH,KLO_Wind_dir,KLO_Wind_dir_delta,KLO_Wind_dir_delta-3,KLO_Wind_dir_delta-6,KLO_Wind_dir_spread30,KLO_Wind_dir_spread60,KLO_Wind_max_EW,KLO_Wind_max_NS,KLO_Wind_max,KLO_Wind_max_delta_3,KLO_Wind_max_delta_6,KLO_Wind_speed_EW,KLO_Wind_speed_NS,KLO_Wind_speed,KLO_Wind_speed_delta_3,KLO_Wind_speed_delta_6,BIZ_Air_temp,BIZ_Hum,BIZ_Hum_delta_3,BIZ_Hum_delta_6,BIZ_QNH,BIZ_Wind_dir,BIZ_Wind_dir_delta,BIZ_Wind_dir_delta-3,BIZ_Wind_dir_delta-6,BIZ_Wind_dir_spread30,BIZ_Wind_dir_spread60,BIZ_Wind_max_EW,BIZ_Wind_max_NS,BIZ_Wind_max,BIZ_Wind_max_delta_3,BIZ_Wind_max_delta_6,BIZ_Wind_speed_EW,BIZ_Wind_speed_NS,BIZ_Wind_speed,BIZ_Wind_speed_delta_3,BIZ_Wind_speed_delta_6,BUS_Air_temp,BUS_Hum,BUS_Hum_delta_3,BUS_Hum_delta_6,BUS_QNH,BUS_Wind_dir,BUS_Wind_dir_delta,BUS_Wind_dir_delta-3,BUS_Wind_dir_delta-6,BUS_Wind_dir_spread30,BUS_Wind_dir_spread60,BUS_Wind_max_EW,BUS_Wind_max_NS,BUS_Wind_max,BUS_Wind_max_delta_3,BUS_Wind_max_delta_6,BUS_Wind_speed_EW,BUS_Wind_speed_NS,BUS_Wind_speed,BUS_Wind_speed_delta_3,BUS_Wind_speed_delta_6,GUT_Air_temp,GUT_Hum,GUT_Hum_delta_3,GUT_Hum_delta_6,GUT_QNH,GUT_Wind_dir,GUT_Wind_dir_delta,GUT_Wind_dir_delta-3,GUT_Wind_dir_delta-6,GUT_Wind_dir_spread30,GUT_Wind_dir_spread60,GUT_Wind_max_EW,GUT_Wind_max_NS,GUT_Wind_max,GUT_Wind_max_delta_3,GUT_Wind_max_delta_6,GUT_Wind_speed_EW,GUT_Wind_speed_NS,GUT_Wind_speed,GUT_Wind_speed_delta_3,GUT_Wind_speed_delta_6,KOP_Air_temp,KOP_Hum,KOP_Hum_delta_3,KOP_Hum_delta_6,KOP_QNH,KOP_Wind_dir,KOP_Wind_dir_delta,KOP_Wind_dir_delta-3,KOP_Wind_dir_delta-6,KOP_Wind_dir_spread30,KOP_Wind_dir_spread60,KOP_Wind_max_EW,KOP_Wind_max_NS,KOP_Wind_max,KOP_Wind_max_delta_3,KOP_Wind_max_delta_6,KOP_Wind_speed_EW,KOP_Wind_speed_NS,KOP_Wind_speed,KOP_Wind_speed_delta_3,KOP_Wind_speed_delta_6,MOA_Air_temp,MOA_Hum,MOA_Hum_delta_3,MOA_Hum_delta_6,MOA_QNH,MOA_Wind_dir,MOA_Wind_dir_delta,MOA_Wind_dir_delta-3,MOA_Wind_dir_delta-6,MOA_Wind_dir_spread30,MOA_Wind_dir_spread60,MOA_Wind_max_EW,MOA_Wind_max_NS,MOA_Wind_max,MOA_Wind_max_delta_3,MOA_Wind_max_delta_6,MOA_Wind_speed_EW,MOA_Wind_speed_NS,MOA_Wind_speed,MOA_Wind_speed_delta_3,MOA_Wind_speed_delta_6,MUB_Air_temp,MUB_Hum,MUB_Hum_delta_3,MUB_Hum_delta_6,MUB_QNH,MUB_Wind_dir,MUB_Wind_dir_delta,MUB_Wind_dir_delta-3,MUB_Wind_dir_delta-6,MUB_Wind_dir_spread30,MUB_Wind_dir_spread60,MUB_Wind_max_EW,MUB_Wind_max_NS,MUB_Wind_max,MUB_Wind_max_delta_3,MUB_Wind_max_delta_6,MUB_Wind_speed_EW,MUB_Wind_speed_NS,MUB_Wind_speed,MUB_Wind_speed_delta_3,MUB_Wind_speed_delta_6,PAY_Air_temp,PAY_Hum,PAY_Hum_delta_3,PAY_Hum_delta_6,PAY_QNH,PAY_Wind_dir,PAY_Wind_dir_delta,PAY_Wind_dir_delta-3,PAY_Wind_dir_delta-6,PAY_Wind_dir_spread30,PAY_Wind_dir_spread60,PAY_Wind_max_EW,PAY_Wind_max_NS,PAY_Wind_max,PAY_Wind_max_delta_3,PAY_Wind_max_delta_6,PAY_Wind_speed_EW,PAY_Wind_speed_NS,PAY_Wind_speed,PAY_Wind_speed_delta_3,PAY_Wind_speed_delta_6,PUY_Air_temp,PUY_Hum,PUY_Hum_delta_3,PUY_Hum_delta_6,PUY_QNH,PUY_Wind_dir,PUY_Wind_dir_delta,PUY_Wind_dir_delta-3,PUY_Wind_dir_delta-6,PUY_Wind_dir_spread30,PUY_Wind_dir_spread60,PUY_Wind_max_EW,PUY_Wind_max_NS,PUY_Wind_max,PUY_Wind_max_delta_3,PUY_Wind_max_delta_6,PUY_Wind_speed_EW,PUY_Wind_speed_NS,PUY_Wind_speed,PUY_Wind_speed_delta_3,PUY_Wind_speed_delta_6,REH_Air_temp,REH_Hum,REH_Hum_delta_3,REH_Hum_delta_6,REH_QNH,REH_Wind_dir,REH_Wind_dir_delta,REH_Wind_dir_delta-3,REH_Wind_dir_delta-6,REH_Wind_dir_spread30,REH_Wind_dir_spread60,REH_Wind_max_EW,REH_Wind_max_NS,REH_Wind_max,REH_Wind_max_delta_3,REH_Wind_max_delta_6,REH_Wind_speed_EW,REH_Wind_speed_NS,REH_Wind_speed,REH_Wind_speed_delta_3,REH_Wind_speed_delta_6,SHA_Air_temp,SHA_Hum,SHA_Hum_delta_3,SHA_Hum_delta_6,SHA_QNH,SHA_Wind_dir,SHA_Wind_dir_delta,SHA_Wind_dir_delta-3,SHA_Wind_dir_delta-6,SHA_Wind_dir_spre

In [0]:
df_merge.shape

(262801, 832)

In [0]:
df_merge.to_csv(join(data_path, MASTER_DATASET), compression='zip')

In [0]:
df_all_features.shape

### Replace dashes with underscores

In [0]:
col_ren = [c for c in df_all_features.columns if re.search('delta-\d$', c)]
renamings = dict()
for c in col_ren:
  renamings[c] = c.replace('-', '_')
df_all_features.rename(columns=renamings, inplace=True)
len([c for c in df_all_features.columns if re.search('delta-\d$', c)])

0

## Wind driection changes along the axis
<ul><li>Wind_dir_split-NS: Wind direction changes along N-S axis (new)</li>
<li>Wind_dir_split--EW: Wind direction changes along E-W axis (new)</li></ul>

In [0]:
def calc_wind_feat_split_ns_ew(df1, dir_col, feat_col):
  
  df = df1.copy()
  lag = None
  """x = re.search('_\d$', dir_col)
  if x:
    lag = x.group()"""
  
  # insert new cols before original 
  ind = df.columns.get_loc(feat_col)
  new_name_ns = feat_col + '_NS' 
  df.insert(ind, new_name_ns, np.NaN)
  df[new_name_ns] = df[feat_col] * np.cos(np.radians(df[dir_col]))
  
  new_name_ew = feat_col + '_EW'
  df.insert(ind, new_name_ew, np.NaN)
  df[new_name_ew] = df[feat_col] * np.sin(np.radians(df[dir_col]))
  
  return df


In [0]:
cols_wind_dir = [c for c in df_all_features.columns if re.search('Wind_dir$', c)]
cols_wind_feat = [c for c in df_all_features.columns if re.search('Wind_max$', c)]

df = df_all_features.copy()
for i, c in enumerate(cols_wind_dir):
  df = calc_wind_feat_split_ns_ew(df, c, cols_wind_feat[i])
  
df.head()
  

ValueError: ignored

In [0]:
df.to_csv(join(data_path, 'model_data_all_features_extended.csv.zip'), compression='zip')
df.shape

(262801, 785)

In [0]:
cols_wind_dir = [c for c in df_all_features.columns if re.search('Wind_dir$', c)]
cols_wind_feat = [c for c in df_all_features.columns if re.search('Wind_speed$', c)]


for i, c in enumerate(cols_wind_dir):
  df = calc_wind_feat_split_ns_ew(df, c, cols_wind_feat[i])
  
print(df.shape)
df.head()

(262801, 817)


,datetime,KLO_Air_temp,KLO_Hum,KLO_Hum_delta_3,KLO_Hum_delta_6,KLO_QNH,KLO_Wind_dir,KLO_Wind_dir_delta,KLO_Wind_dir_delta-3,KLO_Wind_dir_delta-6,KLO_Wind_dir_spread30,KLO_Wind_dir_spread60,KLO_Wind_max_EW,KLO_Wind_max_NS,KLO_Wind_max,KLO_Wind_max_delta_3,KLO_Wind_max_delta_6,KLO_Wind_speed_EW,KLO_Wind_speed_NS,KLO_Wind_speed,KLO_Wind_speed_delta_3,KLO_Wind_speed_delta_6,BIZ_Air_temp,BIZ_Hum,BIZ_Hum_delta_3,BIZ_Hum_delta_6,BIZ_QNH,BIZ_Wind_dir,BIZ_Wind_dir_delta,BIZ_Wind_dir_delta-3,BIZ_Wind_dir_delta-6,BIZ_Wind_dir_spread30,BIZ_Wind_dir_spread60,BIZ_Wind_max_EW,BIZ_Wind_max_NS,BIZ_Wind_max,BIZ_Wind_max_delta_3,BIZ_Wind_max_delta_6,BIZ_Wind_speed_EW,BIZ_Wind_speed_NS,BIZ_Wind_speed,BIZ_Wind_speed_delta_3,BIZ_Wind_speed_delta_6,BUS_Air_temp,BUS_Hum,BUS_Hum_delta_3,BUS_Hum_delta_6,BUS_QNH,BUS_Wind_dir,BUS_Wind_dir_delta,BUS_Wind_dir_delta-3,BUS_Wind_dir_delta-6,BUS_Wind_dir_spread30,BUS_Wind_dir_spread60,BUS_Wind_max_EW,BUS_Wind_max_NS,BUS_Wind_max,BUS_Wind_max_delta_3,BUS_Wind_max_delta_6,BUS_Wind_speed_EW,BUS_Wind_speed_NS,BUS_Wind_speed,BUS_Wind_speed_delta_3,BUS_Wind_speed_delta_6,GUT_Air_temp,GUT_Hum,GUT_Hum_delta_3,GUT_Hum_delta_6,GUT_QNH,GUT_Wind_dir,GUT_Wind_dir_delta,GUT_Wind_dir_delta-3,GUT_Wind_dir_delta-6,GUT_Wind_dir_spread30,GUT_Wind_dir_spread60,GUT_Wind_max_EW,GUT_Wind_max_NS,GUT_Wind_max,GUT_Wind_max_delta_3,GUT_Wind_max_delta_6,GUT_Wind_speed_EW,GUT_Wind_speed_NS,GUT_Wind_speed,GUT_Wind_speed_delta_3,GUT_Wind_speed_delta_6,KOP_Air_temp,KOP_Hum,KOP_Hum_delta_3,KOP_Hum_delta_6,KOP_QNH,KOP_Wind_dir,KOP_Wind_dir_delta,KOP_Wind_dir_delta-3,KOP_Wind_dir_delta-6,KOP_Wind_dir_spread30,KOP_Wind_dir_spread60,KOP_Wind_max_EW,KOP_Wind_max_NS,KOP_Wind_max,KOP_Wind_max_delta_3,KOP_Wind_max_delta_6,KOP_Wind_speed_EW,KOP_Wind_speed_NS,KOP_Wind_speed,KOP_Wind_speed_delta_3,KOP_Wind_speed_delta_6,MOA_Air_temp,MOA_Hum,MOA_Hum_delta_3,MOA_Hum_delta_6,MOA_QNH,MOA_Wind_dir,MOA_Wind_dir_delta,MOA_Wind_dir_delta-3,MOA_Wind_dir_delta-6,MOA_Wind_dir_spread30,MOA_Wind_dir_spread60,MOA_Wind_max_EW,MOA_Wind_max_NS,MOA_Wind_max,MOA_Wind_max_delta_3,MOA_Wind_max_delta_6,MOA_Wind_speed_EW,MOA_Wind_speed_NS,MOA_Wind_speed,MOA_Wind_speed_delta_3,MOA_Wind_speed_delta_6,MUB_Air_temp,MUB_Hum,MUB_Hum_delta_3,MUB_Hum_delta_6,MUB_QNH,MUB_Wind_dir,MUB_Wind_dir_delta,MUB_Wind_dir_delta-3,MUB_Wind_dir_delta-6,MUB_Wind_dir_spread30,MUB_Wind_dir_spread60,MUB_Wind_max_EW,MUB_Wind_max_NS,MUB_Wind_max,MUB_Wind_max_delta_3,MUB_Wind_max_delta_6,MUB_Wind_speed_EW,MUB_Wind_speed_NS,MUB_Wind_speed,MUB_Wind_speed_delta_3,MUB_Wind_speed_delta_6,PAY_Air_temp,PAY_Hum,PAY_Hum_delta_3,PAY_Hum_delta_6,PAY_QNH,PAY_Wind_dir,PAY_Wind_dir_delta,PAY_Wind_dir_delta-3,PAY_Wind_dir_delta-6,PAY_Wind_dir_spread30,PAY_Wind_dir_spread60,PAY_Wind_max_EW,PAY_Wind_max_NS,PAY_Wind_max,PAY_Wind_max_delta_3,PAY_Wind_max_delta_6,PAY_Wind_speed_EW,PAY_Wind_speed_NS,PAY_Wind_speed,PAY_Wind_speed_delta_3,PAY_Wind_speed_delta_6,PUY_Air_temp,PUY_Hum,PUY_Hum_delta_3,PUY_Hum_delta_6,PUY_QNH,PUY_Wind_dir,PUY_Wind_dir_delta,PUY_Wind_dir_delta-3,PUY_Wind_dir_delta-6,PUY_Wind_dir_spread30,PUY_Wind_dir_spread60,PUY_Wind_max_EW,PUY_Wind_max_NS,PUY_Wind_max,PUY_Wind_max_delta_3,PUY_Wind_max_delta_6,PUY_Wind_speed_EW,PUY_Wind_speed_NS,PUY_Wind_speed,PUY_Wind_speed_delta_3,PUY_Wind_speed_delta_6,REH_Air_temp,REH_Hum,REH_Hum_delta_3,REH_Hum_delta_6,REH_QNH,REH_Wind_dir,REH_Wind_dir_delta,REH_Wind_dir_delta-3,REH_Wind_dir_delta-6,REH_Wind_dir_spread30,REH_Wind_dir_spread60,REH_Wind_max_EW,REH_Wind_max_NS,REH_Wind_max,REH_Wind_max_delta_3,REH_Wind_max_delta_6,REH_Wind_speed_EW,REH_Wind_speed_NS,REH_Wind_speed,REH_Wind_speed_delta_3,REH_Wind_speed_delta_6,SHA_Air_temp,SHA_Hum,SHA_Hum_delta_3,SHA_Hum_delta_6,SHA_QNH,SHA_Wind_dir,SHA_Wind_dir_delta,SHA_Wind_dir_delta-3,SHA_Wind_dir_delta-6,SHA_Wind_dir_spread30,SHA_Wind_dir_spread60,SHA_Wind_max_EW,SHA_Wind_max_NS,SHA_Wind_max,SHA_Wind_max_delta_3,SHA_Wind_max_delta_6,SHA_Wind_speed_EW,SHA_Wind_speed_NS,SHA_Wind_speed,SHA_Wind_speed_delta_3,SHA_Wind_speed_delta

In [0]:
print(df_all_features.shape)
df_all_features.to_csv(join(data_path, 'model_data_all_features_extended.csv'))

(262801, 817)


In [0]:
df = pd.DataFrame(columns=['Feature Name'], data=df_all_features.columns)  
df.head()

,Feature Name
0,datetime
1,KLO_Air_temp
2,KLO_Hum
3,KLO_Hum_delta_3
4,KLO_Hum_delta_6


In [0]:
df.to_csv(join(data_path, 'all_features.csv'))